In [1]:
import json
import py.myutils as my
import importlib
importlib.reload(my)
import json
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Patch, Rectangle
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors
import matplotlib.cm as cm
import matplotlib.patheffects as path_effects
from matplotlib.ticker import MaxNLocator
from matplotlib import rc

# plt.rcParams["font.family"] = "serif"
# plt.rcParams["mathtext.fontset"] = "dejavuserif"

rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

recalculate = recompile = True

In [18]:
params = dict(
 b = -0.02,
 c = 1, d = 1, v = -0.1, tau = 1,
    h=0.01, T = 50000
)

script = "ddes_test"
prefix = script
params_str = json.dumps(params)
#
filename = prefix + " " + params_str
filename = filename[:min(len(filename), 200)]

filename_bin = f"output_bin/{filename}.bin"
#
if recalculate or not os.path.isfile(filename_bin):
    my.run_cpp(script, params=params_str, recompile=recompile)



ERROR: compiler exited and returned 256


In file included from /usr/include/c++/11/bits/stl_uninitialized.h:64,
                 from /usr/include/c++/11/vector:66,
                 from cpp/ddes_test.cpp:1:
/usr/include/c++/11/ext/alloc_traits.h: In instantiation of ‘struct __gnu_cxx::__alloc_traits<std::allocator<void>, void>’:
/usr/include/c++/11/bits/stl_vector.h:87:21:   required from ‘struct std::_Vector_base<void, std::allocator<void> >’
/usr/include/c++/11/array:111:56:   recursively required from ‘struct std::array<std::vector<void, std::allocator<void> >, 6>’
/usr/include/c++/11/array:111:56:   required from ‘struct std::array<std::array<std::vector<void, std::allocator<void> >, 6>, 1>’
cpp/discoque.hpp:21:59:   required from ‘class DiscoQue<1, 6, void>’
cpp/ddes.hpp:90:36:   required from ‘void RelayDDE<n, delays_number, true_delays_number>::solution(std::array<double, delays_number>, std::vector<double>&, std::vector<std::array<double, delays_number> >&) [with int n = 2; int delays_number = 1; int true_delays_numb